# Code for loading music datasets
Using preprocessed data downloaded from: http://www-etud.iro.umontreal.ca/~boulanni/icml2012

In [23]:
import pickle
import numpy as np
import torch

In [2]:
path = './'
dataset_name = 'Piano-midi.de'
dataset_path = path + dataset_name

In [3]:
dataset = pickle.load(open(dataset_path + '.pickle', 'rb'))

In [4]:
print('Dataset keys: ', dataset.keys())
print('Test dataset length: ', len(dataset['test']))
print('Example of datapoint = musical piece: ', dataset['test'][0])

Dataset keys:  dict_keys(['test', 'train', 'valid'])
Test dataset length:  25
Example of datapoint = musical piece:  [[36, 48], [36, 48, 60, 64, 67], [31, 43, 60, 64, 67], [36, 48], [36, 48], [36, 48, 60, 64, 67], [31, 43, 60, 64, 67], [48, 55, 60, 64], [48, 55, 60, 64], [55, 65], [43, 64], [48, 55, 60, 64, 67], [48, 55, 60, 64, 67], [55, 60, 64, 67], [43, 64, 72], [48, 64, 76], [48, 64, 76], [55, 64, 72], [49, 63, 69], [50, 59, 65], [50, 59, 65], [55, 59, 65], [43, 62, 65], [47, 65, 69], [47, 65, 69], [55, 71, 74], [43, 69, 72], [50, 71, 74], [50, 67, 71], [55, 67, 71], [43, 71, 74], [47, 69, 72], [47, 69, 72], [55, 67, 71], [43, 62, 65], [48, 63, 69], [48, 64, 67], [55, 64, 67], [55, 60, 72], [45, 52, 60, 72], [45, 52, 60, 69], [45, 51, 60, 65, 71], [45, 51, 60, 65, 69], [45, 60, 72], [45, 60, 64], [60, 64], [52, 64], [45, 60, 65], [45, 60, 72], [60, 71], [53, 69], [45, 60, 64, 72], [52, 60, 64, 72], [57, 60, 64, 72], [60, 66, 76], [59, 67, 76], [56, 65, 71], [57, 66, 72], [60, 66, 6

In [57]:
def chord_to_binary(chord):
    """According to http://www-etud.iro.umontreal.ca/~boulanni/icml2012,
    each chord is a list of the non-zero elements in the piano-roll at this instant.
     (in MIDI note numbers, between 21 and 108 inclusive).
    
    This function transforms the list into a binary vector of length 88 (= 108 - 21 + 1)
    indicating which notes were played in the given chord
    """
    indices = [note - 21 for note in chord]
    binary_vector = torch.zeros(88)
    binary_vector[indices] = 1.0
    return binary_vector

In [58]:
chord = dataset['test'][0][0]
print(chord)
print(chord_to_binary(chord))

[36, 48]
tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.])


In [59]:
def piece_to_binary(piece):
    piece_binary = torch.zeros((len(piece), 88))
    for chord_idx in range(len(piece)):
        piece_binary[chord_idx, :] = chord_to_binary(piece[chord_idx])
    return piece_binary

In [65]:
# Input is binarize piece excluding the final chord
piece_binarized = piece_to_binary(dataset['test'][0])
input = piece_binarized[:-1,:]

# Target is binarize piece excluding the first chord
target = piece_binarized[1:,:]

In [68]:
target[0,:]

tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.])

In [55]:
def log_loss_chord(chord, predicted_chord):
    return (torch.log(1 - predicted_chord).sum() 
            - torch.log(1 - predicted_chord[chord]).sum() 
            + torch.log(predicted_chord[chord]).sum())

In [56]:
predicted_chord = torch.rand((88))
chord = dataset['test'][0][0]
log_loss(chord, predicted_chord)

tensor(-92.4701)

In [92]:
def log_loss_binary_chord(binary_chord, predicted_chord):
    return (binary_chord * torch.log(predicted_chord) 
            + (1-binary_chord) * (torch.log(1-predicted_chord))).sum()

In [95]:
log_loss_binary_chord(target[0,:]+0.01, target[0,:]+0.01)

tensor(nan)

In [107]:
d = 0.0000001
t_d = target[0,:]/(1+d)+d/2
log_loss_binary_chord(t_d, t_d)

tensor(1.00000e-05 *
       -7.9970)

In [77]:
(1-binary_chord) * (1-torch.log(predicted_chord))

tensor([inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf,  0., inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf,  0., inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf,  0., inf, inf, inf,  0., inf, inf,  0., inf,
        inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf])